# Question 1
In the videos, we saw the “diet problem”. (The diet problem is one of the first large-scale optimization problems to be studied in practice. Back in the 1930’s and 40’s, the Army wanted to meet the nutritional requirements of its soldiers while minimizing the cost.) In this homework you get to solve a diet problem with real data. The data is given in the file diet.xls.

Formulate an optimization model (a linear program) to find the cheapest diet that satisfies the maximum and minimum daily nutrition constraints, and solve it using PuLP. Turn in your code and the solution. (The optimal solution should be a diet of air-popped popcorn, poached eggs, oranges, raw iceberg lettuce, raw celery, and frozen broccoli. UGH!)

### Importing and formatting data
To start off, we'll be using the pulp library along with the pandas library in this homework assignment.  Two primary arrays were created: constraints which contain the constraint criterias for the problem and data which contains the nutritional content of each food item.  

In [1]:
from pulp import *
import pandas as pd

raw = pd.read_excel('diet.xls')

#pull constraint info out
constraints=raw.iloc[65:67,2:raw.shape[1]] 

#remove serving size since it serves no purpose
raw = raw.drop('Serving Size', 1) 

#convert to list for easier dictionary creation
data=raw[0:64].values.tolist() 

### Create Nutrition Dictionary
Next we can manually create a column for each nutritional value.  However, if this list gets extremely big, it would be a waste of time to manually generate this.  I created a for loop that loops through each of the individual nutrition column and generated a dictionary for each with respect to the food item that the nutrition is associated with.  All of this is stored in the attr variable.

In [2]:
foods=[x[0] for x in data]
attr=[]
for i in range(len(data[1])-1):
    if i==0:
        attr.append([x[0] for x in data])
    else:
        attr.append(dict([(x[0],float(x[i])) for x in data]))

### Variable
Now we can establish the problem as a minimization function to minimize the costand to declare our variable foodVars.

In [3]:
prob=LpProblem("Food optimization", LpMinimize)
foodVars=LpVariable.dicts("Foods",attr[0],0)

### Objective Function
This is the fucntion that we are trying to solve for minimizing the total cost.

In [4]:
prob += lpSum([attr[1][f] * foodVars[f] for f in attr[0]]), 'Total Cost'

### Constraints
Because we've stored both the nutritional content and the constraints in arrays, we can easily loop through them instead of manually writing them out.  We can also automatically name each of the constraints in the for loop

In [5]:
for j in range(2,12):
    minstr="min "+constraints.columns[j]
    maxstr="max "+constraints.columns[j]
    prob += lpSum([attr[j][f] * foodVars[f] for f in attr[0]]) \
            >= constraints.iloc[0,j-1], minstr
    prob += lpSum([attr[j][f] * foodVars[f] for f in attr[0]]) \
            <= constraints.iloc[1,j-1], maxstr

### Solve 
Here the writeLP function can be used to copy all of our constraints into a .lp file in our python directory for us to view.  Once we've solved the optimization problem, we can see what it has chosen for us...wayyyy too much vegetables and not a single "real" meat dish...I'm disappointed

In [6]:
prob.writeLP("xyz.lp")
prob.solve()
for var in prob.variables():
    if var.varValue >0:
        print(str(var.varValue)+" units of "+var.name)

print ("Total Cost of ingredients = ", value(prob.objective))

52.64371 units of Foods_Celery,_Raw
0.25960653 units of Foods_Frozen_Broccoli
63.988506 units of Foods_Lettuce,Iceberg,Raw
2.2929389 units of Foods_Oranges
0.14184397 units of Foods_Poached_Eggs
13.869322 units of Foods_Popcorn,Air_Popped
Total Cost of ingredients =  4.337116797399999


# Question 2
Please add to your model the following constraints (which might require adding more variables) and solve the new model: 

* If a food is selected, then a minimum of 1/10 serving must be chosen. (Hint: now you will need two variables for each food i: whether it is chosen, and how much is part of the diet. You’ll also need to write a constraint to link them.)
* Many people dislike celery and frozen broccoli. So at most one, but not both, can be selected. 
* To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be selected.

### Adding Constraints
Question 2 asked us to add additional constraints to this problem.  Since we've already set the constraints from before, all we need to do is to just add to it.  But before we do that, we also need to add an additional variable of food_select to help us determine whether a food item has been chosen or not.  This can be a binary value.  Next we'll add all of our constraints.

In [7]:
#Establish the food select variable to determine if the item was chosen or not
food_select=LpVariable.dicts("food_select",foods,0,1,LpBinary)

#Atleast 1/10 of a server if the food is selected
for food in foods:
    prob += foodVars[food] >= 0.1*food_select[food]

for food in foods:
    prob += food_select[food] >= foodVars[food]*0.0000001

#Either Celery or Broccoli
prob += food_select['Frozen Broccoli'] + food_select['Celery, Raw'] <= 1

#Atleast 3 different Meat, Poultry, Egg, or Fish
prob += food_select['Roasted Chicken'] + \
food_select['Poached Eggs'] + \
food_select['Scrambled Eggs'] + \
food_select['Bologna,Turkey'] + \
food_select['Frankfurter, Beef'] + \
food_select['Ham,Sliced,Extralean'] + \
food_select['Hamburger W/Toppings'] + \
food_select['Hotdog, Plain'] + \
food_select['Pork'] + \
food_select['Sardines in Oil'] + \
food_select['White Tuna in Water'] + \
food_select['Chicknoodl Soup'] + \
food_select['Splt Pea&Hamsoup'] + \
food_select['Vegetbeef Soup'] + \
food_select['Neweng Clamchwd'] + \
food_select['New E Clamchwd,W/Mlk'] + \
food_select['Beanbacn Soup,W/Watr'] >=3

### Solve
After solving with these additional constraints, it's finally a relief to see that we have Turkey and two different types of eggs in our meal now!  Although, we did increase the amount of Iceberg lettuce.  I guess, Skyrim's got the right idea!

In [8]:
prob.solve()
for var in prob.variables():
    if var.varValue >0 and "food_select" not in var.name:
        print(str(var.varValue)+" units of "+var.name)

print ("Total Cost of ingredients = ", value(prob.objective))

0.1 units of Foods_Bologna,Turkey
42.423026 units of Foods_Celery,_Raw
82.673927 units of Foods_Lettuce,Iceberg,Raw
3.0856009 units of Foods_Oranges
1.9590978 units of Foods_Peanut_Butter
0.1 units of Foods_Poached_Eggs
13.214473 units of Foods_Popcorn,Air_Popped
0.1 units of Foods_Scrambled_Eggs
Total Cost of ingredients =  4.5129554810000005


# Homework Extras
If you want to see what a more full-sized problem would look like, try solving the model of Question 1 for the file diet_large.xls, which is a low-cholesterol diet model (rather than minimizing cost, the goal is to minimize cholesterol intake). I don’t know anyone who’d want to eat this diet – the optimal solution includes dried chrysanthemum garland, raw beluga whale flipper, freeze-dried parsley, etc. – which shows why it’s necessary to add additional constraints beyond the basic ones we saw in the video!

### Response
Since I generalized my code previously, I was also able to apply that to this problem, making it simpler to work with.  I had to make a few tweaks to variable indicies but overall it's replicating what I've done above.

In [9]:
%reset -f
from pulp import *
import pandas as pd
rawlarge = pd.read_excel('diet_large.xls',header=1)
rawlarge = rawlarge.fillna(0)

#pull constraint info out
constraints=rawlarge.iloc[[7147,7149],1:28]

#convert to list for easier dictionary creation
datadf=rawlarge.iloc[0:7146,0:29]
data=rawlarge.iloc[0:7146,0:29].values.tolist() 

In [10]:
# Create Nutrition Dictionary
foods=[x[0] for x in data]
attr=[]
for i in range(len(data[1])):
    if i==0:
        attr.append([x[0] for x in data])
    else:
        attr.append(dict([(x[0],float(x[i])) for x in data]))

# Variables
prob=LpProblem("Food optimization", LpMinimize)
foodVars=LpVariable.dicts("Foods",attr[0],0)

# Objective Function
prob += lpSum([attr[28][f] * foodVars[f] for f in attr[0]]), 'Total Cholesterol'

# Constraints
for j in range(1,28):
    minstr="min "+constraints.columns[j-1]
    maxstr="max "+constraints.columns[j-1]
    prob += lpSum([attr[j][f] * foodVars[f] for f in attr[0]]) \
            >= constraints.iloc[0,j-1], minstr
    prob += lpSum([attr[j][f] * foodVars[f] for f in attr[0]]) \
            <= constraints.iloc[1,j-1], maxstr

In [11]:
# Solve
prob.writeLP("abc.lp")
prob.solve()
for var in prob.variables():
    if var.varValue >0:
        print(str(var.varValue)+" units of "+var.name)

print ("Total cholesterol of ingredients = ", value(prob.objective))

0.059863415 units of Foods_Beans,_adzuki,_mature_seeds,_raw
0.069514608 units of Foods_Broccoli_raab,_raw
0.42866218 units of Foods_Cocoa_mix,_no_sugar_added,_powder
0.14694398 units of Foods_Egg,_white,_dried,_flakes,_glucose_reduced
0.73805891 units of Foods_Infant_formula,_MEAD_JOHNSON,_ENFAMIL,_NUTRAMIGEN,_with_iron,_p
0.4258564 units of Foods_Infant_formula,_NESTLE,_GOOD_START_ESSENTIALS__SOY,__with_iron,
0.050114149 units of Foods_Infant_formula,_ROSS,_ISOMIL,_with_iron,_powder,_not_reconstitu
0.15033656 units of Foods_Margarine_like_spread,_approximately_60%_fat,_tub,_soybean_(hyd
0.25918767 units of Foods_Mung_beans,_mature_seeds,_raw
0.18052856 units of Foods_Nuts,_mixed_nuts,_dry_roasted,_with_peanuts,_with_salt_added
1.184482 units of Foods_Oil,_vegetable,_sunflower,_linoleic,_(hydrogenated)
0.10375187 units of Foods_Seeds,_sunflower_seed_kernels,_dry_roasted,_with_salt_added
0.031866196 units of Foods_Snacks,_potato_chips,_fat_free,_made_with_olestra
0.070710308 units of Fo

On thing I noticed is that this isn't the same as what was provided in the answers.  However, I did also achieve a cholesterol level of 0.0.  It may be possible that the solver may be different than the one that Professor Sokol used  but still found this as an optimal solution.  